# CP 2 Aprendizaje de Máquinas
---
## Clasificación supervisada e introducción a la herramienta sklearn

El objetivo de la clase práctica es resolver un problema utilizando los algoritmos dados en conferencia. Se tienen críticas de cine positivas y negativas. 1000 de cada una. El objetivo es, dado el texto de la crítica, determinar si la crítica es positiva o negativa.

### Ejercicio 1: Extracción de los datos

En la carpeta `txt_sentoken` se encuentran las críticas. Dentro hay dos carpetas (`pos`, `neg`), donde están las críticas positivas y negativas.

In [3]:
path_p = u"txt_sentoken/pos"
path_n = u"txt_sentoken/neg"

Cada archivo de cada carpeta es una crítica diferente, por lo que tenemos que analizar los archivos de cada una de estas carptetas para procesar las críticas.

In [4]:
import os
ds_p = os.listdir(path_p)
ds_n = os.listdir(path_n)

Luego, cada archivo tenemos que convertirlos en cadenas de texto.

In [5]:
def convert_file_to_text(name: str):
    text = u""
    f = open(name)
    for i in f.readlines():
        text += i
    return text
        

Por ejemplo, de esta forma se convierte un archivo:

In [6]:
a = convert_file_to_text(os.path.join(path_p, ds_p[0]))
a

'films adapted from comic books have had plenty of success , whether they\'re about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there\'s never really been a comic book like from hell before . \nfor starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid \'80s with a 12-part series called the watchmen . \nto say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd . \nthe book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes . \nin other words , don\'t dismiss this film because of its source . \nif you can get past the whole comic book thing , you might find another stumbling block in from hell\'s directors , albert and allen hughes . \ngetting the hughes brothers to direct this seem

Luego, hacemos lo mismo para cada archivo, guardándolos en dos listas diferentes, una de críticas positivas y otra de críticas negativas.

In [7]:
texts_p = []
texts_n = []

In [8]:
for file in ds_p:
    texts_p.append(convert_file_to_text(os.path.join(path_p, file)))

In [9]:
for file in ds_n:
    texts_n.append(convert_file_to_text(os.path.join(path_n, file)))

Para comprobar el resultado de este proceso, se comprueba la longitud de las listas

In [10]:
len(texts_p)

1000

In [11]:
len(texts_n)

1000

### Ejercicio 2: Extracción de características

¿Cómo convertimos una cadena de texto en una matriz de características?

Existen varias maneras, una de las más sencillas, es usando `CountVectorizer` de `sklearn`, que convierte una collección de documentos en una matriz con recuentos de tokens.

In [12]:
from sklearn.feature_extraction.text import CountVectorizer 

In [13]:
vectorizer = CountVectorizer()
mt = vectorizer.fit_transform(texts_p + texts_n)
mta = mt.toarray()

La matriz de características tiene dos dimensiones, la primera representa la cantidad de instancias (ejemplos) y la segunda la cantidad de características. 

In [14]:
# matriz de características
mt.shape

(2000, 39659)

¿Cuál es el por ciento de los valores que son distintos de cero? 

In [15]:
mt.nnz *100.0 / (mt.shape[0] * mt.shape[1])

0.8407196348874152

El vector de clase dice, para cada instancia cuál es la clase a la que pertenece. La clase positiva (1) representa una crítica positiva y la clase negativa (0) una crítica negativa.

In [16]:
y = [1]*1000 + [0]*1000

### Ejercicio 3: Elección del clasificador

Ahora que ya se tienen la matriz de clasificación y el vector de clases, ¿qué hacemos?, ¿qué clasificador usamos?, ¿por qué?

El Naive Bayes de sklearn se llama Gaussian Naive Bayes porque está diseñado (a diferencia del visto en conferencia) para lidiar con características que sean valores continuos. En vez de tener probabilidad del feature dada la clase que es $P(w|c)$ ahora lo que se tiene
es la probabilidad de observar un valor tan alto como $x_i$, esto es $P(x \leq x_i | c)$


In [17]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

### Ejercicio 4: Entrenamiento de los algoritmos

Para comenzar el entrenamiento del clasificador elegido, es necesario dividir la matriz de características y el vector de clase en dos conjuntos, uno de entrenamiento y otro de prueba.

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mta, y, train_size=0.60)

Se comprueba que el 60% del total de instancias (1000 ejemplos) correponden al conjunto de entrenamiento (serían 1200 instancias) y el resto (800) al conjunto de prueba.  

In [19]:
print(X_train.shape)
print(X_test.shape)

(1200, 39659)
(800, 39659)


Primero, se realiza el entrenamiento con Naive Bayes con todos los datos, y se prueban los resultados obtenidos en este mismo conjunto.

In [20]:
naive_bayes = GaussianNB()
naive_bayes.fit(mta,y)
naive_bayes.score(mta,y)

0.9985

Ahora, se hace correctamente, realizando el entrenamiento en el conjunto de entrenamiento y probando los resultados obtenidos en un conjunto de prueba, que nunca antes fue visto por el algoritmo.

In [21]:
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)
naive_bayes.score(X_test, y_test)

0.65125

### Ejercicio 5: Probar con el resto de los clasificadores vistos: KNN

Ahora probemos los resultados con el otro algoritmo dado en clases, KNN.

In [22]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

0.6625

### Ejercicio 6: Experimentos

Para comprobar la eficiencia real del algoritmo es necesario realizar el entrenamiento y prueba del algoritmo utilizado varias veces. Así, se tiene un valor más estadísticamente significativo del rendimiento del algoritmo en el dataset utilizado.

In [23]:
def experiments(Clsf, iterations: int):
    rs = []
    for _ in range(iterations):
        X_train, X_test, y_train, y_test = train_test_split(mta, y, train_size=0.60)
        clf = Clsf()
        clf.fit(X_train, y_train)
        rs.append(clf.score(X_test, y_test))
    return rs

In [24]:
import numpy as np

Se comprueban los resultados obtenidos con el algoritmo de KNN.

In [30]:
rs1 = experiments(KNeighborsClassifier, 30)
np.mean(rs1)

0.6274583333333333

Y luego se comprueban los resultados con Naive Bayes.

In [31]:
rs2 = experiments(GaussianNB, 30)
np.mean(rs2)

0.6478750000000001

Por último, comprobamos mediante una prueba de hipótesis si la media de las muestras correspondientes a los resultados de rendimiento son diferentes.

In [33]:
from scipy.stats import ttest_ind
result = ttest_ind(rs1, rs2)
result.pvalue < 0.01

True